## Predict

In [1]:
import os
from glob import glob

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import datasets

import matplotlib.pyplot as plt

%matplotlib inline

## Hyper parameter Tunning

In [2]:
num_epochs = 10 
batch_size = 32
learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

## Build Model

In [3]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(32, (3,3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3,3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2,2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3,3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3,3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2,2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs = inputs, outputs = net)
model.summary()

model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate),
             loss = 'sparse_categorical_crossentropy',
             metrics = ['accuracy'])

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 32)        0     

## Data Process

In [4]:
## (1) train, test 데이터 읽어오기
print(os.listdir('../dataset/cifar/'))

train_paths = glob('../dataset/cifar/train/*.png')[:1000]
test_paths = glob('../dataset/cifar/test/*.png')[:1000]

print(len(train_paths), len(test_paths))

## (2) label name 반환
def get_class_name(path):
    return path.split('_')[-1].replace('.png','')

## (3) label name 고유값 list - class_names
train_labels = [get_class_name(path) for path in train_paths] # 50000개
class_names = np.unique(train_labels) # 해당 class 개로 줄여짐

# (4) label 반환 onehot encoding 된 형태로 or argmax 로 해당 index
def get_label(path):
    fname = tf.strings.split(path, '_')[-1]
    lbl_name = tf.strings.regex_replace(fname, '.png', '')
    onehot = tf.cast(lbl_name == class_names, tf.uint8)
    return tf.argmax(onehot) 

# (5) 최종 image, label 반환
def load_image_label(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    image = tf.cast(image, tf.float32)/.255 # rescale
    
    label = get_label(path)
    return image, label

# (6) image preprocess (랜덤 Generator)
def image_preprocess(image, label):
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_flip_left_right(image)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE # 작동하는 Network가 스스로 설정하고 Dataset을 잘 불러올 수 있게 결정하라는 것


# train dataset 뽑기
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(load_image_label, num_parallel_calls = AUTOTUNE)
train_dataset = train_dataset.map(image_preprocess, num_parallel_calls = AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size = len(train_paths))
train_dataset = train_dataset.repeat()

# test set 데이터 뽑기
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()
      

['labels.txt', 'test', 'train']
1000 1000


## Training

In [5]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

history = model.fit_generator(
    train_dataset,
    steps_per_epoch = steps_per_epoch,
    validation_data = test_dataset,
    validation_steps = validation_steps,
    epochs = num_epochs,
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
31/31 [==============================] - 10s 318ms/step - loss: 37.1151 - accuracy: 0.0950 - val_loss: 2.2973 - val_accuracy: 0.1230
Epoch 2/10
31/31 [==============================] - 11s 349ms/step - loss: 2.3343 - accuracy: 0.1095 - val_loss: 2.2997 - val_accuracy: 0.1159
Epoch 3/10
31/31 [==============================] - 9s 280ms/step - loss: 2.3157 - accuracy: 0.1116 - val_loss: 2.2994 - val_accuracy: 0.1200
Epoch 4/10
31/31 [==============================] - 7s 213ms/step - loss: 2.3032 - accuracy: 0.1200 - val_loss: 2.2998 - val_accuracy: 0.1290
Epoch 5/10
31/31 [==============================] - 6s 180ms/step - loss: 2.3044 - accuracy: 0.1133 - val_loss: 2.2980 - val_accuracy: 0.1310
Epoch 6/10
31/31 [==============================] - 6s 181ms/step - loss: 2.3040 - accuracy: 0.1220 - val_loss: 2.3001 - val_accuracy: 0.1069
Epoch 7/10
31/31 [==============================] - 6s 186ms/step - l

## Predict
- 방법1 이미지를 Load 직접 Load 해서 넣는 방법
- 방법2 generator에서 데이터를 가져오는 방법
- 방법3 generator. take 로  넣는 방법  


In [8]:
# 방법 1 이미지를 직접 Load 해서 넣는 방법

# image 1장 가져옴.
path = test_paths[0]

gfile = tf.io.read_file(path)
image = tf.io.decode_image(gfile, dtype=tf.float32)

# image 4차원으로 늘려줌
image = image[tf.newaxis, ...]
print(image.shape)

pred = model.predict(image)
print(pred)
print(np.argmax(pred))

(1, 32, 32, 3)
[[0.09860038 0.09980676 0.08974691 0.09830822 0.10321076 0.09639487
  0.10063316 0.11138067 0.10345271 0.09846556]]
7


In [10]:
## 방법2 generator에서 데이터를 가져오는 방법
# 차원수를 직접 늘리거나 필요가 없음
test_image, test_label = next(iter(test_dataset))
print(test_image.shape)

pred2 = model.predict(test_image)
print(pred2[0].shape) # label 10개 - 첫번째 이미지
print(np.argmax(pred2[0]))

(32, 32, 32, 3)
(10,)
7


In [14]:
## 방법3 generaotr에 넣는 방법  
pred3 = model.predict_generator(test_dataset.take(3))
print(pred3.shape)
print(np.argmax(pred3[0]))

(96, 10)
7
